In [1]:
# -----------------------------------------------------------
# Import Required Libraries
# -----------------------------------------------------------

import re
import pandas as pd
from bs4 import BeautifulSoup, NavigableString
from datetime import datetime

In [7]:
# -----------------------------------------------------------
# Define Final Robust Text-Based Parser
# -----------------------------------------------------------

def parse_patch_notes_final(filepath):
    '''
    Reads the Diablo IV patch notes HTML file.
    Extracts ordered text nodes directly.
    Uses regex-based state tracking for:
        - Patch
        - Section
        - Subsection (Class/System)
        - Category
        - Previous/Now comparisons
    Returns structured DataFrame.
    '''

    # -----------------------------------------------------------
    # STEP 1: Read HTML
    # -----------------------------------------------------------

    with open(filepath, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")


    # -----------------------------------------------------------
    # STEP 2: Define Patterns & Headers
    # -----------------------------------------------------------

    patch_pattern = re.compile(
        r"(\d+\.\d+\.\d+)\s+Build\s+#(\d+).*?([A-Za-z]+\s+\d{1,2},\s+\d{4})"
    )

    section_headers = {
        "Bug Fixes", "Game Updates", "Balance Update",
        "Balance Updates", "Expansion", "Base Game",
        "Accessibility", "Gameplay", "Miscellaneous",
        "Activities", "Enemy Updates", "Economy Updates",
        "Monster Updates", "Nightmare Dungeons",
        "Masterworking", "Tempering", "Toughness",
        "Season Rank"
    }

    subsection_headers = {
        "Barbarian", "Druid", "Sorcerer",
        "Rogue", "Necromancer",
        "Spiritborn", "Paladin",
        "All Classes", "Mercenaries",
        "Kurast Undercity", "Base Game"
    }

    category_headers = {
        "Skills", "Passives", "Items",
        "Legendary Aspects", "Paragon",
        "Enchantments", "Unique Items",
        "Affixes", "Power"
    }


    # -----------------------------------------------------------
    # STEP 3: Initialize State
    # -----------------------------------------------------------

    records = []

    current_patch = None
    current_build = None
    current_date = None

    current_section = None
    current_subsection = None
    current_category = None

    previous_buffer = None


    # -----------------------------------------------------------
    # STEP 4: Iterate Text Nodes in Order
    # -----------------------------------------------------------

    for element in soup.descendants:

        if not isinstance(element, NavigableString):
            continue

        text = element.strip()

        if not text:
            continue


        # --- Detect Patch Header ---
        patch_match = patch_pattern.search(text)

        if patch_match:
            current_patch = patch_match.group(1)
            current_build = patch_match.group(2)
            current_date = patch_match.group(3)

            current_section = None
            current_subsection = None
            current_category = None
            previous_buffer = None

            continue


        # Skip everything before first patch header
        if current_patch is None:
            continue


        # --- Detect Section ---
        if text in section_headers:
            current_section = text
            current_subsection = None
            current_category = None
            continue


        # --- Detect Subsection ---
        if text in subsection_headers:
            current_subsection = text
            current_category = None
            continue


        # --- Detect Category ---
        if text in category_headers:
            current_category = text
            continue


        # --- Detect Previous ---
        if text.startswith("Previous"):
            previous_buffer = text.replace("Previous:", "").strip()
            continue


        # --- Detect Now ---
        if text.startswith("Now") and previous_buffer:
            now_text = text.replace("Now:", "").strip()

            records.append({
                "patch": current_patch,
                "build": current_build,
                "date": current_date,
                "section": current_section,
                "subsection": current_subsection,
                "category": current_category,
                "change_type": "comparison",
                "previous": previous_buffer,
                "now": now_text,
                "full_text": f"Changed from {previous_buffer} to {now_text}"
            })

            previous_buffer = None
            continue


        # --- Regular Content Row ---
        records.append({
            "patch": current_patch,
            "build": current_build,
            "date": current_date,
            "section": current_section,
            "subsection": current_subsection,
            "category": current_category,
            "change_type": "single",
            "previous": None,
            "now": None,
            "full_text": text
        })


    # -----------------------------------------------------------
    # STEP 5: Build DataFrame
    # -----------------------------------------------------------

    df = pd.DataFrame(records)


    # -----------------------------------------------------------
    # STEP 6: Basic Noise Cleanup
    # -----------------------------------------------------------

    noise_patterns = [
        "Patch Notes",
        "Diablo IV",
        "Blizzard Entertainment",
        "Do Not Sell",
        "Your Privacy",
        "Required Cookies",
        "Allow All"
    ]

    df = df[~df["full_text"].isin(noise_patterns)]

    df = df[
        ~df["full_text"].str.contains(
            "Privacy|Cookies|Consent|Share Or Sale",
            case=False,
            na=False
        )
    ]

    df = df.reset_index(drop=True)

    return df

In [8]:
# -----------------------------------------------------------
# Run Final Parser
# -----------------------------------------------------------

input_file = "Diablo_IV_Patch_Notes.html"

df_final = parse_patch_notes_final(input_file)

print("✅ Final Parsing Complete.")
print("Total records extracted:", len(df_final))

✅ Final Parsing Complete.
Total records extracted: 2206


In [9]:
df_final.head(25)

,patch,build,date,section,subsection,category,change_type,previous,now,full_text
0,2.5.3,70356,"January 28, 2026",Bug Fixes,NaN,NaN,single,NaN,NaN,Fixed an issue where the Executioner Monster A...
1,2.5.3,70356,"January 28, 2026",Bug Fixes,NaN,NaN,single,NaN,NaN,Developer’s Note:
2,2.5.3,70356,"January 28, 2026",Bug Fixes,NaN,NaN,single,NaN,NaN,This affix will be re-enabled with the release...
3,2.5.3,70356,"January 28, 2026",Bug Fixes,NaN,NaN,single,NaN,NaN,Fixed an issue where certain Silent Chests in ...
4,2.5.3,70356,"January 28, 2026",Bug Fixes,NaN,NaN,single,NaN,NaN,Fixed an issue where Zagraal in the Dark Citad...
5,2.5.3,70356,"January 28, 2026",Bug Fixes,NaN,NaN,single,NaN,NaN,Fixed an issue where some Tower bosses had sig...
6,2.5.3,70356,"January 28, 2026",Bug Fixes,NaN,NaN,single,NaN,NaN,Fixed an issue where an error would occur when...
7,2.5.3,70356,"January 28, 2026",Bug Fixes,NaN,NaN,single,NaN,NaN,Fixed an issue where resetting a piece of mast...
8,2.5.3,70356,"January 28, 2026",Bug Fixes,NaN,NaN,single,NaN,NaN,Fixed an issue where the reward for defeating ...
9,2.5.3,70356,"January 28, 2026",Bug Fixes,NaN,NaN,single,NaN,NaN,Fixed an issue where other Divine Gifts could ...


In [14]:
df_final.to_csv("diablo_iv_patch_notes_structured.csv", index=False)
print("✅ CSV file saved.")

✅ CSV file saved.


In [16]:
import requests

url = "https://news.blizzard.com/en-us/article/24244466/diablo-iv-patch-notes"
response = requests.get(url)
html = response.text

with open("clean_patch_notes.html", "w", encoding="utf-8") as f:
    f.write(html)